Replicating the Stage 1 Leaderboard Score
===============================================
To ensure full understanding of the objective function in relation to the actual games, I wanted stand-alone code that can match a sample submission.

## Inputs
- The data from the 2022 Men's Competition
- sub5.csv: a non-trivial submission

## Goal
To match the submission score 0.54253 that was computed automatically.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
sub_df = pd.read_csv('../input/sub5csv/sub5.csv')
# Nothing special about this submission, except that it's non-trivial (not all .5s)
sub_df['Season'] = sub_df.ID.apply(lambda s: int(s.split("_")[0]))
sub_df['TeamIDLower'] = sub_df.ID.apply(lambda s: int(s.split("_")[1]))
sub_df['TeamIDUpper'] = sub_df.ID.apply(lambda s: int(s.split("_")[2]))
sub_df.head()

In [ ]:
tourney_df = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
tourney_df.head()

In [ ]:
lower_wins = tourney_df.loc[(tourney_df.WTeamID < tourney_df.LTeamID)].copy()
upper_wins = tourney_df.loc[(tourney_df.WTeamID > tourney_df.LTeamID)].copy()
assert(lower_wins.shape[0] + upper_wins.shape[0] == tourney_df.shape[0])

lower_wins = lower_wins.rename({'WTeamID': 'TeamIDLower', 'LTeamID': 'TeamIDUpper'}, axis=1)[['Season', 'TeamIDLower', 'TeamIDUpper']]
lower_wins['Winner'] = 'lower'

upper_wins = upper_wins.rename({'WTeamID': 'TeamIDUpper', 'LTeamID': 'TeamIDLower'}, axis=1)[['Season', 'TeamIDLower', 'TeamIDUpper']]
upper_wins['Winner'] = 'upper'

outcomes_df = pd.concat([lower_wins, upper_wins])
outcomes_df.head()

In [ ]:
aug_df = sub_df.merge(outcomes_df, on=['Season', 'TeamIDLower', 'TeamIDUpper'])
aug_df.head()

In [ ]:
aug_df['logloss'] = (aug_df['Winner'] == 'lower') * np.log(aug_df['Pred']) + (aug_df['Winner'] == 'upper') * np.log(1 - aug_df['Pred'])
aug_df.head()

In [ ]:
# This is not quite matching 0.54253
-np.mean(aug_df['logloss'])

In [ ]:
aug_df[['ID', 'Pred']].to_csv('submission.csv', index=False)  # See if I can get the notebook to show a score